In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#라이브러리

In [2]:
## install
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-m_kyfgb2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-m_kyfgb2
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=26842e5f3d34cb252cc17e397a26b8a90e592ffd1373447cd4bdc28ec8f75a05
  Stored in directory: /tmp/pip-ephem-wheel-cache-81z6a7rr/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depe

In [3]:
!pip install ffmpeg-python

In [4]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 45.4 MB/s eta 0:00:00


In [5]:
import whisper
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
from pprint import pprint
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
import tempfile
import wave
import subprocess

from urllib.error import HTTPError
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
from konlpy.tag import Okt

import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

okt = Okt()
path = '/content/gdrive/MyDrive/4학년 2학기/창종설/data/'

df_common_term = pd.read_csv(path + '/csv/common_term.csv')

model_whisper = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 93.1MiB/s]


# whisper

In [26]:
stop_words = ['당한', '것']

def stop_words_func(text_list):
  text_stopwords = []
  for text in text_list:
      word_tokens = okt.morphs(text)
      if not text in stop_words:
        text_stopwords.append(text)
      # # 불용어가 제거된 형태소 분석 결과 출력
      # result = ' '.join(text_stopwords)
      # text_stopwords.append(result)

  return text_stopwords

In [27]:
def whisper_stt():

  def get_audio():

    AUDIO_HTML = """
    <script>
    var my_div = document.createElement("DIV");
    var my_p = document.createElement("P");
    var my_btn = document.createElement("BUTTON");
    var t = document.createTextNode("Press to start recording");

    my_btn.appendChild(t);
    //my_p.appendChild(my_btn);
    my_div.appendChild(my_btn);
    document.body.appendChild(my_div);

    var base64data = 0;
    var reader;
    var recorder, gumStream;
    var recordButton = my_btn;

    var handleSuccess = function(stream) {
      gumStream = stream;
      var options = {
        //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
        mimeType : 'audio/webm;codecs=opus'
        //mimeType : 'audio/webm;codecs=pcm'
      };
      //recorder = new MediaRecorder(stream, options);
      recorder = new MediaRecorder(stream);
      recorder.ondataavailable = function(e) {
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data);
        reader.onloadend = function() {
          base64data = reader.result;
          //console.log("Inside FileReader:" + base64data);
        }
      };
      recorder.start();
      };

    recordButton.innerText = "Recording... press to stop";

    navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


    function toggleRecording() {
      if (recorder && recorder.state == "recording") {
          recorder.stop();
          gumStream.getAudioTracks()[0].stop();
          recordButton.innerText = "Saving the recording... pls wait!"
      }
    }

    // https://stackoverflow.com/a/951057
    function sleep(ms) {
      return new Promise(resolve => setTimeout(resolve, ms));
    }

    var data = new Promise(resolve=>{
    //recordButton.addEventListener("click", toggleRecording);
    recordButton.onclick = ()=>{
    toggleRecording()

    sleep(2000).then(() => {
      // wait 2000ms for the data to be available...
      // ideally this should use something like await...
      //console.log("Inside data:" + base64data)
      resolve(base64data.toString())

    });

    }
    });

    </script>
    """

    display(HTML(AUDIO_HTML))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])

    process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
    )
    output, err = process.communicate(input=binary)

    riff_chunk_size = len(output) - 8
    # Break up the chunk size into four bytes, held in b.
    q = riff_chunk_size
    b = []
    for i in range(4):
        q, r = divmod(q, 256)
        b.append(r)

    # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
    riff = output[:4] + bytes(b) + output[8:]

    sr, audio = wav_read(io.BytesIO(riff))

    return audio, sr


  # 음성을 받아서 텍스트로 처리하는 함수
  def whisper_model(audio):

      # load audio and pad/trim it to fit 30 seconds
      audio = whisper.load_audio(audio)
      audio = whisper.pad_or_trim(audio)

      # make log-Mel spectrogram and move to the same device as the model
      mel = whisper.log_mel_spectrogram(audio).to(model_whisper.device)

      # detect the spoken language
      _, probs = model_whisper.detect_language(mel)
      # print(f"Detected language: {max(probs, key=probs.get)}")

      # decode the audio
      options = whisper.DecodingOptions()
      result = whisper.decode(model_whisper, mel, options)
      return result.text


  def convert_wav_to_mp3(input_wav, output_mp3):
    subprocess.call(['ffmpeg', '-i', input_wav, output_mp3])


  def audio_to_tempfile(audio_data, sampling_rate):
    # 임시 WAV 파일에 오디오 데이터를 저장합니다.
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
        temp_filename = temp_file.name
        with wave.open(temp_filename, 'wb') as wave_file:
            wave_file.setnchannels(1)
            wave_file.setsampwidth(2)
            wave_file.setframerate(sampling_rate)
            wave_file.writeframes(audio_data)

        # convert_wav_to_mp3 함수를 사용하여 WAV 파일을 MP3 파일로 변환합니다.
        temp_mp3_file = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
        temp_mp3_file = temp_mp3_file.name
        convert_wav_to_mp3(temp_filename, temp_mp3_file)

    return(temp_filename)


    #음성 인식
  audio_data, sampling_rate = get_audio()

  audio_file_mp3 = audio_to_tempfile(audio_data, sampling_rate)

  #받은 데이터
  result = whisper_model(audio_file_mp3)

  noun_list = okt.nouns(result)

  noun_result = stop_words_func(noun_list)

  # noun_list = okt.nouns(result)

  # noun_result = ' '.join(noun_list)

  return result, noun_result

In [28]:
from subprocess import run, PIPE
from google.colab import files

def whisper_stt_read():
  def audio_to_tempfile(audio_data, sampling_rate):
      # 임시 WAV 파일에 오디오 데이터 저장
      with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
          temp_filename = temp_file.name
          with wave.open(temp_filename, 'wb') as wave_file:
              wave_file.setnchannels(1)
              wave_file.setsampwidth(2)
              wave_file.setframerate(sampling_rate)
              wave_file.writeframes(audio_data)

      return temp_filename

  def whisper_model(audio):

      # load audio and pad/trim it to fit 30 seconds
      audio = whisper.load_audio(audio)
      audio = whisper.pad_or_trim(audio)

      # make log-Mel spectrogram and move to the same device as the model
      mel = whisper.log_mel_spectrogram(audio).to(model_whisper.device)

      # detect the spoken language
      _, probs = model_whisper.detect_language(mel)
      # print(f"Detected language: {max(probs, key=probs.get)}")

      # decode the audio
      options = whisper.DecodingOptions()
      result = whisper.decode(model_whisper, mel, options)
      return result.text

  def get_audio():
      # 파일 업로드 함수
      uploaded_files = files.upload()
      if not uploaded_files:
          print("파일이 업로드되지 않았습니다.")
          return None

      # 업로드한 파일 확인
      file_names = list(uploaded_files.keys())
      file_contents = uploaded_files[file_names[0]]

      # 업로드한 파일을 ffmpeg를 사용하여 WAV로 변환
      process = run(['ffmpeg', '-i', '-', '-f', 'wav', '-'], input=file_contents, stdout=PIPE, stderr=PIPE)
      wav_data = process.stdout

      sr, audio = wav_read(io.BytesIO(wav_data))
      return audio, sr

  audio_data, sampling_rate = get_audio()

  audio_file_mp3 = audio_to_tempfile(audio_data, sampling_rate)

  #받은 데이터
  result = whisper_model(audio_file_mp3)


  noun_list = okt.nouns(result)

  noun_result = stop_words_func(noun_list)

  # noun_list = okt.nouns(result)

  # noun_result = ' '.join(noun_list)


  return result, noun_result

#법령-일상 연계

In [8]:
def legal_common_integration(noun_result):

  term_indices = []
  term_legal = []

  for keyword in noun_result:
    term_legal.append(keyword)

    # indices_containing_word = df_common_term[df_common_term['term_name'].str.contains(keyword)].index

    for index, row in df_common_term.iterrows():
        if row['term_name'] == keyword:
            term_indices.append(index)
    print(term_indices)

    # 찾은 인덱스를 사용하여 'B' 열의 값을 출력
    for idx in term_indices:
      url_df = df_common_term.at[idx, 'term_link']
      url = 'https://www.law.go.kr' + url_df

      print(url)

      try:
          result = urlopen(url)
      except HTTPError as e:
          term_legal.append(df_common_term.at[idx, 'term_name'])
          # error_list.append([index, url])
          # print("HTTP error")
          continue  # HTTP 오류가 발생하면 다음 반복으로 넘어감

      house = BeautifulSoup(result, 'lxml-xml')
      te = house.find_all('연계용어')

      check_num = 0
      for i in range(len(te)):
          legal_term = te[i].법령용어명.string.strip()
          term_legal.append(legal_term)
          check_num += 1
          if check_num ==2:
            break

  return term_legal

# transformer

In [9]:
# 리스트 불러오기
with open(path + 'input_text_violent.json', 'r') as f:
    input_text_violent = json.load(f)

with open(path + 'output_text_violent.json', 'r') as f:
    output_text_violent = json.load(f)

# 서브워드텍스트인코더를 사용하여 질문과 답변을 모두 포함한 단어 집합(Vocabulary) 생성
tokenizer_violent = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    input_text_violent + output_text_violent, target_vocab_size=2**13)

# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN_violent, END_TOKEN_violent = [tokenizer_violent.vocab_size], [tokenizer_violent.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE_violent = tokenizer_violent.vocab_size + 2

# 리스트 불러오기
with open(path + 'input_text_scam.json', 'r') as f:
    input_text_scam = json.load(f)

with open(path + 'output_text_scam.json', 'r') as f:
    output_text_scam = json.load(f)

# 서브워드텍스트인코더를 사용하여 질문과 답변을 모두 포함한 단어 집합(Vocabulary) 생성
tokenizer_scam = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    input_text_scam + output_text_scam, target_vocab_size=2**13)

# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN_scam, END_TOKEN_scam = [tokenizer_scam.vocab_size], [tokenizer_scam.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE_scam = tokenizer_scam.vocab_size + 2

In [10]:
MAX_LENGTH = 500

class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    # print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

  # def get_config(self):
  #   base_config = super().get_config()
  #   config = {
  #       "position": tf.keras.saving.serialize_keras_object(self.position),
  #   }
  #   return {**base_config, **config}

  # def get_config(self):

  #   config = super(PositionalEncoding, self).get_config()
  #   config.update({
  #       'position': self.position,
  #       'd_model': self.d_model,

  #   })
  #   return config

class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

  # def get_config(self):
  #       config = {
  #           'd_model':self.d_model,
  #           'warmup_steps':self.warmup_steps

  #       }
  #       base_config = super(CustomSchedule, self).get_config()
  #       return dict(list(base_config.items()) + list(config.items()))

  def get_config(self):
    config = {
    'd_model': self.d_model,
    'warmup_steps': self.warmup_steps,

     }
    return config

# def evaluate(model, sentence):
#   sentence = preprocess_sentence(sentence)

#   sentence = tf.expand_dims(
#       START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

#   output = tf.expand_dims(START_TOKEN, 0)

#   # 디코더의 예측 시작
#   for i in range(MAX_LENGTH):
#     predictions = model(inputs=[sentence, output], training=False)

#     # 현재(마지막) 시점의 예측 단어를 받아온다.
#     predictions = predictions[:, -1:, :]
#     predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

#     # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
#     if tf.equal(predicted_id, END_TOKEN[0]):
#       break

#     # 마지막 시점의 예측 단어를 출력에 연결한다.
#     # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
#     output = tf.concat([output, predicted_id], axis=-1)

#   return tf.squeeze(output, axis=0)


# def predict(model, sentence):
#   prediction = evaluate(model, sentence)

#   predicted_sentence = tokenizer.decode(
#       [i for i in prediction if i < tokenizer.vocab_size])

#   print('Input: {}'.format(sentence))
#   print('Output: {}'.format(predicted_sentence))

#   return predicted_sentence

def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [11]:
# Hyper-parameters
NUM_LAYERS = 2 # 2
D_MODEL = 256 # 256
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model_violent = transformer(
    vocab_size=VOCAB_SIZE_violent,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model_violent.load_weights(path + 'model_violent_weights').expect_partial()

model_scam = transformer(
    vocab_size=VOCAB_SIZE_scam,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model_scam.load_weights(path + 'model_scam_weights').expect_partial()


In [15]:
def transformer_violent(term_list):

  def evaluate(model, sentence):
    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(
        START_TOKEN_violent + tokenizer_violent.encode(sentence) + END_TOKEN_violent, axis=0)

    output = tf.expand_dims(START_TOKEN_violent, 0)

    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
      predictions = model(inputs=[sentence, output], training=False)

      # 현재(마지막) 시점의 예측 단어를 받아온다.
      predictions = predictions[:, -1:, :]
      predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

      # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
      if tf.equal(predicted_id, END_TOKEN_violent[0]):
        break

      # 마지막 시점의 예측 단어를 출력에 연결한다.
      # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
      output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)

  def predict(model, sentence):
    prediction = evaluate(model, sentence)

    predicted_sentence = tokenizer_violent.decode(
        [i for i in prediction if i < tokenizer_violent.vocab_size])

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

  result_term = ' '.join(term_list)

  output = predict(model_violent, result_term)

  return output

In [16]:
def transformer_scam(term_list):

  def evaluate(model, sentence):
    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(
        START_TOKEN_scam + tokenizer_scam.encode(sentence) + END_TOKEN_scam, axis=0)

    output = tf.expand_dims(START_TOKEN_scam, 0)

    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
      predictions = model(inputs=[sentence, output], training=False)

      # 현재(마지막) 시점의 예측 단어를 받아온다.
      predictions = predictions[:, -1:, :]
      predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

      # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
      if tf.equal(predicted_id, END_TOKEN_scam[0]):
        break

      # 마지막 시점의 예측 단어를 출력에 연결한다.
      # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
      output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0)

  def predict(model, sentence):
    prediction = evaluate(model, sentence)

    predicted_sentence = tokenizer_scam.decode(
        [i for i in prediction if i < tokenizer_scam.vocab_size])

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

  result_term = ' '.join(term_list)

  output = predict(model_scam, result_term)

  return output

# 실행

In [34]:
# 폭행1
print("다음 보기 중 문의하려는 키워드의 번호를 선택해주세요.")

print("1. 폭행")
print("2. 사기")
print("\n")

user_input = input("번호: ")

# result, noun_result = whisper_stt()
result, noun_result = whisper_stt_read()

print("말씀하신 문장은 다음과 같습니다")

print('"' + result + '"')

if user_input == "1":
  output = transformer_violent(noun_result)
elif user_input =="2":
  output = transformer_scam(noun_result)

다음 보기 중 문의하려는 키워드의 번호를 선택해주세요.
1. 폭행
2. 사기


번호: 1


Saving 녹음_폭행1.wav to 녹음_폭행1 (3).wav


<ipython-input-28-30b9ab9c941c>:50: WavFileWarning: Reached EOF prematurely; finished at 352369 bytes, expected 4294967303 bytes from header.
  sr, audio = wav_read(io.BytesIO(wav_data))


말씀하신 문장은 다음과 같습니다
"성추행과 강간을 당했습니다."
Input: 성추행 강간
Output: 성폭력범죄의처벌등에관한특례법위반


In [35]:
# 폭행2
print("다음 보기 중 문의하려는 키워드의 번호를 선택해주세요.")

print("1. 폭행")
print("2. 사기")
print("\n")

user_input = input("번호: ")

# result, noun_result = whisper_stt()
result, noun_result = whisper_stt_read()

print("말씀하신 문장은 다음과 같습니다")

print('"' + result + '"')

if user_input == "1":
  output = transformer_violent(noun_result)
elif user_input =="2":
  output = transformer_scam(noun_result)

다음 보기 중 문의하려는 키워드의 번호를 선택해주세요.
1. 폭행
2. 사기


번호: 1


Saving 녹음_폭행2.wav to 녹음_폭행2 (1).wav


<ipython-input-28-30b9ab9c941c>:50: WavFileWarning: Reached EOF prematurely; finished at 604273 bytes, expected 4294967303 bytes from header.
  sr, audio = wav_read(io.BytesIO(wav_data))


말씀하신 문장은 다음과 같습니다
"술집에서 술을 마시다가 쌍방폭행으로 싸움이 일어났습니다."
Input: 술집 술 쌍방 폭행 싸움
Output: 폭력행위등처벌에관한법률위반


In [36]:
# 사기1
print("다음 보기 중 문의하려는 키워드의 번호를 선택해주세요.")

print("1. 폭행")
print("2. 사기")
print("\n")

user_input = input("번호: ")

# result, noun_result = whisper_stt()
result, noun_result = whisper_stt_read()

print("말씀하신 문장은 다음과 같습니다")

print('"' + result + '"')

if user_input == "1":
  output = transformer_violent(noun_result)
elif user_input =="2":
  output = transformer_scam(noun_result)

다음 보기 중 문의하려는 키워드의 번호를 선택해주세요.
1. 폭행
2. 사기


번호: 2


Saving 녹음_사기1.wav to 녹음_사기1 (4).wav


<ipython-input-28-30b9ab9c941c>:50: WavFileWarning: Reached EOF prematurely; finished at 579693 bytes, expected 4294967303 bytes from header.
  sr, audio = wav_read(io.BytesIO(wav_data))


말씀하신 문장은 다음과 같습니다
"부동산 증서를 양도하여 계약하였는데 사기를 당한 것 같아요"
Input: 부동산 증서 양도 계약 사기
Output: 사문서위조 사문서위조행사 공정증서원본불실기재 동행사 사기 횡령
